In [123]:
import collections
import random

In [124]:
PIPELINE_WIDTH = 2
NUM_RESERVATION_STATIONS = 4
NUM_ROB_ENTRIES = 8
NUM_REGISTERS = 32
INSTRUCTION_QUEUE_SIZE = 32
NUM_RAT_ENTRIES = 32
opcodes = {
    0x0 : 'rtype', 0x8: 'addi', 0xd :'ori', 0xe: 'xori', 0xc:'andi', 0xa:'slti',
    0x23: 'lw', 0x2b : 'sw', 0x4:'beq', 0x2:'j', 0x3 : 'jal', 0x5:'bne'
      
}
notWrites = [0x2b, 0x4, 0x2, 0x5]
immediates = [0x8, 0xd, 0xe, 0xc, 0xa]
rtypes = {
    0x20: 'add', 0x22:'sub', 0x24:'and', 0x25: 'or', 0x2a:'slt', 0x26: 'xor', 
    0x27:'nor', 0x0:'sll', 0x2:'srl', 0x8:'jr'  
}
def signed(data):
        if data & 0x8000:
            return data - (1 << 16)  
        else: return data

In [125]:
instructions = []
with open("imem.txt", 'r') as inst:
    for line in inst:
        parts = line.split(":")
        if len(parts)>1:
            instructions.append(int(parts[1].strip().rstrip(";"),2))
    

In [126]:
class Instruction:
    def __init__(self, instruction):
        self.instructionVal = bin(instruction)
        self.opcode = instruction >> 26 & 0b111111
        self.rs = instruction >> 21 & 0b11111
        self.rt = instruction >> 16 & 0b11111
        self.rd = instruction >> 11 & 0b11111
        self.validRt = opcodes[self.opcode] == 'rtype' or opcodes[self.opcode] =='bne' or opcodes[self.opcode] == 'beq' #Check if not immediate or not. If immediate the val is false
        self.destreg = self.rd if self.validRt else self.rt
        self.shamt = instruction >> 6 & 0b11111
        self.funct = instruction & 0b111111
        self.imm = instruction & 0xFFFF

In [127]:
class ROBEntry:
    def __init__(self, instruction):
        self.instruction = instruction
        self.value = None
        self.ready = False
        self.tag = 0
        self.destination = instruction.destreg

In [128]:
class ReservationStation:
    def __init__(self):
        self.instruction = None
        self.tag_dest = None
        self.tag_src1 = None
        self.tag_src2 = None
        self.ready = 0
        self.value_src1 = None
        self.value_src2 = None
        self.busy = False

In [129]:
class RAT:
    def __init__(self):
        self.rename = 0
        self.allocated = 0

In [130]:
class PipelineRegister:
    def __init__(self):
        self.flush = 0
        self.instruction = 0
        self.instructionVal = 0
        self.prediction = 0
        self.stateIndex = 0
        self.last = 0
        self.data = 0 # any data, depending on the stage. signals are interpreted directly w/o a CU so no need for multiple signals 
        self.done = 0
        self.flush = 0
        self.pc = 0

In [131]:
instruction_memory = []


class ProcessorState:
    def __init__(self):
        self.cycle = 0
        self.pc = 0
        self.registers = [0]*32
        self.instructionsQueue = collections.deque()
        self.rob = collections.deque()
        self.reservation_stations = [ReservationStation() for _ in range(NUM_RESERVATION_STATIONS)]
        self.memory = [0] * 1024
        self.rat = [RAT() for _ in range(NUM_RAT_ENTRIES)]
        self.rsfull = 0

In [132]:
state = ProcessorState()
state.memory[0] = 0x5
state.memory[1] = 0x7
state.memory[2] = 0x2
state.memory[3] = 0xF
state.memory[4] = 0xA
state.memory[5] = 0x10
state.memory[6] = 0x30
state.memory[7] = 0x1
state.memory[8] = 0xFF
state.memory[9] = 0x55

In [133]:

def fetch():
    inst  = Instruction(0)
    if state.pc<len(instructions):
        inst = Instruction(instructions[state.pc])
    state.instructionsQueue.append(inst)
    


In [170]:
def dispatch():
   if not state.rsfull and len(state.instructionsQueue) > 0:
      inst = state.instructionsQueue.popleft()
      state.rob.append(ROBEntry(inst))
      tag = len(state.rob) - 1
      state.rob[-1].tag = tag 
      state.rat[inst.destreg].rename = tag 
      state.rat[inst.destreg].allocated = 1
      for i in range(0, 4):
         if not state.reservation_stations[i].busy:
            state.reservation_stations[i].instruction = inst
            state.reservation_stations[i].tag_dest = tag
            state.reservation_stations[i].ready = 0
            if not state.rat[inst.rs].allocated:
               state.reservation_stations[i].value_src1 = state.registers[inst.rs]
               state.reservation_stations[i].ready = state.reservation_stations[i].ready | 0b01
            else:
               state.reservation_stations[i].tag_src1 = state.rat[inst.rs].rename
            if not state.rat[inst.rt].allocated or not inst.validRt:
               state.reservation_stations[i].value_src2 = state.registers[inst.rt] if inst.validRt else inst.imm
               state.reservation_stations[i].ready = state.reservation_stations[i].ready | 0b10
            else:
               state.reservation_stations[i].tag_src2 = state.rat[inst.rt].rename
            state.reservation_stations[i].busy = True
            break
   for k in range(0, 4):
      for x in range(0, NUM_ROB_ENTRIES):
         if x < len(state.rob):  
            if (state.reservation_stations[k].busy) and not (state.reservation_stations[k].ready & 0b1) and (state.reservation_stations[k].tag_src1 == state.rob[x].tag) and state.rob[x].ready:
                state.reservation_stations[k].value_src1 = state.rob[x].value
                state.reservation_stations[k].ready = state.reservation_stations[k].ready | 0b01
            if (state.reservation_stations[k].busy) and not (state.reservation_stations[k].ready >> 1 & 0b1) and (state.reservation_stations[k].tag_src2 == state.rob[x].tag) and state.rob[x].ready:
                state.reservation_stations[k].value_src2 = state.rob[x].value
                state.reservation_stations[k].ready = state.reservation_stations[k].ready | 0b10
   if state.reservation_stations[0].busy and state.reservation_stations[1].busy and state.reservation_stations[2].busy and state.reservation_stations[3].busy:
         state.rsfull = 1


In [171]:
def execute():
    for i in range(0, 2):
        for j in range(0, 4):
            if state.reservation_stations[j].ready == 0b11 and state.reservation_stations[j].busy:
                inst = state.reservation_stations[j].instruction
                op1 = state.reservation_stations[j].value_src1
                op2 = state.reservation_stations[j].value_src2
                state.reservation_stations[j].busy = 0
                state.reservation_stations[j].ready = 0
                if opcodes[inst.opcode] == 'rtype':
                    if rtypes[inst.funct] =='add':
                        data = op1 + op2
                    elif rtypes[inst.funct] =='sub':
                        data = op1 - op2
                    elif rtypes[inst.funct] =='and':
                        data = op1 & op2
                    elif rtypes[inst.funct] =='or':
                        data = op1 | op2
                    elif rtypes[inst.funct] =='slt':
                        data = 1 if op1<op2 else 0
                    elif rtypes[inst.funct] =='xor':
                        data = op1 ^ op2
                    elif rtypes[inst.funct] =='nor':
                        data = ~(op1 | op2)
                    elif rtypes[inst.funct] =='sll':
                        data = op1 <<inst.shamt
                    elif rtypes[inst.funct] =='srl':
                        data = op1 >> inst.shamt
                elif opcodes[inst.opcode] == 'addi':
                    data = op1+op2
                elif opcodes[inst.opcode] == 'ori':
                    data = op1|op2
                elif opcodes[inst.opcode] == 'xori':
                    data = op1^op2
                elif opcodes[inst.opcode] == 'andi':
                    data = op1&op2
                elif opcodes[inst.opcode] == 'slti':
                    data = 1 if op1<op2 else 0
                if len(state.rob) > 0:
                    for x in range(0, NUM_ROB_ENTRIES):
                        if x < len(state.rob):
                            if state.rob[x].tag == state.reservation_stations[j].tag_dest:
                                state.rob[x].ready = 1
                                state.rob[x].value = data

In [180]:

def commit():
    for i in range(0, 2):
        if len(state.rob) > 0:
            if state.rob[0].ready:
                robentry = state.rob.popleft()
                state.rat[robentry.instruction.destreg].rename = 0
                state.rat[robentry.instruction.destreg].allocated = 0
                if robentry.instruction.destreg != 0:
                    state.registers[robentry.instruction.destreg] = robentry.value 

In [181]:
def print_processor_state(proc_state):
    print("\n" + "=" * 60)
    print(f"Cycle: {proc_state.cycle} | PC: {proc_state.pc}")
    print("=" * 60)

    # Print Register Alias Table (RAT)
    print("\n[Register Alias Table (RAT)]")
    for i, rat_entry in enumerate(proc_state.rat):
        print(f"R{i}: Rename={rat_entry.rename} | Allocated={rat_entry.allocated}")

    # Print Reservation Stations
    print("\n[Reservation Stations]")
    for i, rs in enumerate(proc_state.reservation_stations):
        if rs.busy:
            print(f"RS[{i}]: Instr={rs.instruction} | DestTag={rs.tag_dest} | "
                  f"Src1Tag={rs.tag_src1}, Src2Tag={rs.tag_src2} | "
                  f"Src1Val={rs.value_src1}, Src2Val={rs.value_src2} | "
                  f"Busy={rs.busy}, Ready={rs.ready}")

    # Print Reorder Buffer (ROB)
    print("\n[Reorder Buffer (ROB)]")
    for i, rob_entry in enumerate(proc_state.rob):
        print(f"ROB[{i}]: Instr={rob_entry.instruction} | Dest={rob_entry.destination} | "
              f"Tag={rob_entry.tag} | Value={rob_entry.value} | Ready={rob_entry.ready}")

    # Print Instruction Queue
    print("\n[Instruction Queue]")
    for i, inst in enumerate(proc_state.instructionsQueue):
        print(f"Instr[{i}]: {inst.instructionVal}")

    # Print Registers
    print("\n[Registers]")
    for i in range(0, len(proc_state.registers), 8):  # Print in chunks of 8 for better readability
        print(" ".join(f"R{i+j}: {proc_state.registers[i+j]:<5}" for j in range(8) if i+j < len(proc_state.registers)))

    # Optional: Print Memory Snapshot (Uncomment if needed)
    # print("\n[Memory Snapshot]")
    # for addr in range(0, len(proc_state.memory), 8):  # Display in groups of 8
    #     print(f"Mem[{addr:04d}]:", proc_state.memory[addr:addr+8])

    print("=" * 60)


In [182]:
cycles = 50
state.pc = 0
state.cycle = 0
state = ProcessorState()
for i in range(0, cycles):
    commit()
    execute()
    dispatch()
    fetch()
    print_processor_state(state)
    state.pc += 1
    state.cycle += 1
    
    


Cycle: 0 | PC: 0

[Register Alias Table (RAT)]
R0: Rename=0 | Allocated=0
R1: Rename=0 | Allocated=0
R2: Rename=0 | Allocated=0
R3: Rename=0 | Allocated=0
R4: Rename=0 | Allocated=0
R5: Rename=0 | Allocated=0
R6: Rename=0 | Allocated=0
R7: Rename=0 | Allocated=0
R8: Rename=0 | Allocated=0
R9: Rename=0 | Allocated=0
R10: Rename=0 | Allocated=0
R11: Rename=0 | Allocated=0
R12: Rename=0 | Allocated=0
R13: Rename=0 | Allocated=0
R14: Rename=0 | Allocated=0
R15: Rename=0 | Allocated=0
R16: Rename=0 | Allocated=0
R17: Rename=0 | Allocated=0
R18: Rename=0 | Allocated=0
R19: Rename=0 | Allocated=0
R20: Rename=0 | Allocated=0
R21: Rename=0 | Allocated=0
R22: Rename=0 | Allocated=0
R23: Rename=0 | Allocated=0
R24: Rename=0 | Allocated=0
R25: Rename=0 | Allocated=0
R26: Rename=0 | Allocated=0
R27: Rename=0 | Allocated=0
R28: Rename=0 | Allocated=0
R29: Rename=0 | Allocated=0
R30: Rename=0 | Allocated=0
R31: Rename=0 | Allocated=0

[Reservation Stations]

[Reorder Buffer (ROB)]

[Instruction Queu